# Data Preparation

In [ ]:
from __future__ import absolute_import
from numpy import asarray, save
from mqt.predictor import utils, ml

import numpy as np
np.random.seed(10)

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

predictor = ml.Predictor()

(X_train,
X_test,
y_train,
y_test,
indices_train,
indices_test,
names_list,
scores_list) = predictor.get_prepared_training_data(save_non_zero_indices=True)

scores_filtered = [scores_list[i] for i in indices_test]
names_filtered = [names_list[i] for i in indices_test]

performance = []

In [ ]:
print(len(X_train), len(X_test))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(random_state=0)

tree_param = [
    {
        "n_estimators": [100, 200, 500],
        "max_depth": list(range(8, 30, 6)),
        "min_samples_split": list(range(2, 20, 6)),
        "min_samples_leaf": list(range(2, 20, 6)),
        "bootstrap": [True, False],
    },
]

clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)


y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="RandomForestClassifier")
rel_goodness = np.round(np.mean(rel_scores),4)
rel_goodness_std = np.round(np.std(rel_scores),4)
print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Random Forest", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

In [ ]:
predictor.plot_eval_all_detailed_compact_normed(
    names_filtered, scores_filtered, y_pred, y_test
)

### Feature Importances

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

path = ml.helper.get_path_trained_model() / "non_zero_indices.npy"
non_zero_indices = np.load(str(path), allow_pickle=True)
        
openqasm_qc_list = ml.helper.get_openqasm_gates()
feature_names = [openqasm_qc_list[i] for i in range(0, len(openqasm_qc_list))]
feature_names.append("num_qubits")
feature_names.append("depth")
feature_names.append("program_communication")
feature_names.append("critical_depth")
feature_names.append("entanglement_ratio")
feature_names.append("parallelism")
feature_names.append("liveness")
feature_names = [feature_names[i] for i in non_zero_indices]

importances = clf.best_estimator_.feature_importances_
std = np.std(
    [tree.feature_importances_ for tree in clf.best_estimator_.estimators_], axis=0
)

idx = np.argsort(-importances)

plt.figure(figsize=(8, 8))
plt.bar(np.array(feature_names)[idx], np.array(importances)[idx])
plt.errorbar(
    np.array(feature_names)[idx],
    np.array(importances)[idx],
    np.array(std)[idx],
    fmt="o",
    color="#ff8600",
)
plt.xticks(rotation=90, fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel("Relative feature importance", fontsize=18)
plt.tight_layout()
plt.savefig("results/feature_importances.pdf")
plt.show()

#### Check the relative importances per feature

In [ ]:
summary = zip(np.array(feature_names)[idx], np.array(importances)[idx])
for feature, importance in list(summary):
    print(feature, np.round(importance,3))

# GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()

param_grid = {
    "learning_rate": [0.01, 0.1, 1],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))

res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="GradientBoostingClassifier")
rel_goodness = np.round(np.mean(rel_scores),4)
rel_goodness_std = np.round(np.std(rel_scores),4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Gradient Boosting", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Decision Tree Classifier

In [ ]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

clf = tree.DecisionTreeClassifier(random_state=5)

tree_param = [
    {
        "criterion": ["entropy", "gini"],
        "max_depth": list(range(1, 15, 1)),
        "min_samples_split": list(range(2, 20, 4)),
        "min_samples_leaf": list(range(2, 20, 4)),
        "max_leaf_nodes": list(range(2, 200, 40)),
        "max_features": list(range(1, len(non_zero_indices), 10)),
    },
]
clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)
y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="DecisionTreeClassifier")
rel_goodness = np.round(np.mean(rel_scores),4)
rel_goodness_std = np.round(np.std(rel_scores),4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
performance.append(("Decision Tree", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
param_grid = dict(n_neighbors=range(1, 10, 1))
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="KNeighborsClassifier")
rel_goodness = np.round(np.mean(rel_scores),4)
rel_goodness_std = np.round(np.std(rel_scores),4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Nearest Neighbor", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(max_iter=1000)

param_grid = {
    "hidden_layer_sizes": [(50, 50, 50), (50, 100, 50), (100,)],
    "activation": ["tanh", "relu"],
    "solver": ["sgd", "adam"],
    "alpha": [0.0001, 0.05],
    "learning_rate": ["constant", "adaptive"],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="MLPClassifier")
rel_goodness = np.round(np.mean(rel_scores),4)
rel_goodness_std = np.round(np.std(rel_scores),4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Multilayer Perceptron", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# SVM

In [ ]:
from sklearn import svm

clf = svm.SVC()
param_grid = {"C": [0.1, 1, 10], "gamma": [1, 0.1, 0.01], "kernel": ["rbf", "sigmoid"]}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="SVM")
rel_goodness = np.round(np.mean(rel_scores),4)
rel_goodness_std = np.round(np.std(rel_scores),4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Support Vector Machine", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
param_grid = {"var_smoothing": np.logspace(0, -9, num=100)}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res, rel_scores = predictor.calc_performance_measures(scores_filtered, y_pred, y_test)
predictor.plot_eval_histogram(res, filename="GaussianNB")
rel_goodness = np.round(np.mean(rel_scores),4)
rel_goodness_std = np.round(np.std(rel_scores),4)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Rel Goodness: ", rel_goodness)
print("Rel Goodness Std: ", rel_goodness_std)
performance.append(("Naive Bayes", clf.best_score_, top3, max(res), rel_goodness, rel_goodness_std))

# Save Performance Results

In [ ]:
print(performance)

filename = "results/performances.csv"
with open(filename, "w") as f:
    f.write("Classifier, Accuracy, Top3, Worst Rank, Eval. Score Diff., Std\n")
    for sublist in performance:
        line = "{}, {}, {}, {}, {}, {} \n".format(
            sublist[0], sublist[1], sublist[2], sublist[3], sublist[4], sublist[5]
        )
        f.write(line)